In [1]:
###Importing Important Libraries

import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import os
import random

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
###Unzipping the Data
import zipfile

##Train
local_zip_train = '/kaggle/input/aerial-cactus-identification/train.zip'
zip_ref = zipfile.ZipFile(local_zip_train, 'r')
zip_ref.extractall('/kaggle/working')
zip_ref.close()

##Test
local_zip_test = '/kaggle/input/aerial-cactus-identification/test.zip'
zip_ref = zipfile.ZipFile(local_zip_test, 'r')
zip_ref.extractall('/kaggle/working')
zip_ref.close()

In [3]:
###Image Hyperparameters

IMAGE_SIZE = 150
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE,IMAGE_SIZE
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

In [4]:
cd '/kaggle/input/aerial-cactus-identification'

/kaggle/input/aerial-cactus-identification


In [5]:
df = pd.read_csv('train.csv')
df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [6]:
df["has_cactus"] = df["has_cactus"].replace({0: 'No', 1: 'Yes'}) 

In [7]:
df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,Yes
1,000c8a36845c0208e833c79c1bffedd1.jpg,Yes
2,000d1e9a533f62e55c289303b072733d.jpg,Yes
3,0011485b40695e9138e92d0b3fb55128.jpg,Yes
4,0014d7a11e90b62848904c1418fc8cf2.jpg,Yes


In [8]:
###Making the Model
from keras.applications import VGG16

input_shape = (IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_CHANNELS)

model = Sequential

pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")

for layer in pre_trained_model.layers[:15]:
    layer.trainable = False
for layer in pre_trained_model.layers[15:]:
    layer.trainable = True

last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = Flatten()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

58892288/58889256 [==============================] - 2s 0us/step


In [9]:
###Compiling the Model

model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])

In [10]:
###Model Summary

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [17]:
###Callbacks

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

earlystop = EarlyStopping(patience=10)

callbacks = [earlystop, learning_rate_reduction]

In [18]:
###Preparing the Data

train_df, validate_df = train_test_split(df, test_size=0.10, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [19]:
###Hyperparameters


total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=64
epochs = 15

In [20]:
###Generator

##Train
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "/kaggle/working/train", 
    x_col='id',
    y_col='has_cactus',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=batch_size
)

Found 15750 validated image filenames belonging to 2 classes.


In [21]:

##Val 
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/kaggle/working/train", 
    x_col='id',
    y_col='has_cactus',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=batch_size
)

Found 1750 validated image filenames belonging to 2 classes.


In [16]:
###Training the Model

history = model.fit_generator(
    train_generator, 
    epochs=5,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/5
246/246 [==============================] - 87s 354ms/step - loss: 0.1808 - accuracy: 0.9269 - val_loss: 0.0917 - val_accuracy: 0.9705
Epoch 2/5
  1/246 [..............................] - ETA: 22s - loss: 0.0389 - accuracy: 0.9688

/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


246/246 [==============================] - 85s 345ms/step - loss: 0.0673 - accuracy: 0.9783 - val_loss: 0.0258 - val_accuracy: 0.9846
Epoch 3/5
246/246 [==============================] - 84s 343ms/step - loss: 0.0451 - accuracy: 0.9836 - val_loss: 0.0437 - val_accuracy: 0.9858
Epoch 4/5
246/246 [==============================] - 85s 345ms/step - loss: 0.0477 - accuracy: 0.9836 - val_loss: 0.0552 - val_accuracy: 0.9887
Epoch 5/5
246/246 [==============================] - 86s 349ms/step - loss: 0.0553 - accuracy: 0.9840 - val_loss: 0.0178 - val_accuracy: 0.9911


In [22]:
###Testing

##Creating a test DataFrame
filenames_test = os.listdir("/kaggle/working/test")
print(len(filenames_test))

df_test = pd.DataFrame({
    'id': filenames_test,
})

4000


In [23]:
###Creating the Test Image Generator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator=test_datagen.flow_from_dataframe(
dataframe=df_test,
directory="/kaggle/working/test",
x_col="id",
y_col=None,
batch_size=32,
shuffle=False,
class_mode=None,
target_size=(IMAGE_HEIGHT,IMAGE_WIDTH))

Found 4000 validated image filenames.


In [24]:
###Predicting
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

125/125 [==============================] - 6s 46ms/step


In [25]:
pred = np.where(pred > 0.5, 1, 0)

In [34]:
cd '/kaggle/working'

/kaggle/working


In [35]:
prediction = df_test
prediction['label'] = pred
prediction.to_csv('Prediction.csv') 